### Предварительные сведения

Векторные нормы:

$||u||_{\infty} = \max_i|u_i|$

$||u||_1 = \sum_i |u_i|$

$||u||_2 = \left(\sum_i |u_i|^2 \right)^{\frac{1}{2}}$

Матричные нормы:

$||A||_{\infty} = \max_i \sum_j |a_{ij}|$

$||A||_1 = \max_j \sum_i |a_{ij}|$

$||A||_2 = \left(\max_i \lambda_i(A A^*) \right)^{\frac{1}{2}}$

Контрольный вопрос: какова будет вторая норма матрицы, если матрица самосопряженная?

Ваш ответ на контрольный вопрос: вторая норма матрицы будет равна модулю максимального собственного значения этой матрицы

In [1]:
import numpy as np
import numpy.linalg as la

A = np.array([[1,2],[3,4]])
v = range(0,3)
Vander = np.vander(v)
print('norm_1 = ', la.norm(Vander, 1))
print('norm_2 = ', la.norm(Vander, 2))
print('norm_inf = ', la.norm(Vander, np.inf))
Vander

norm_1 =  5.0
norm_2 =  4.844958524498339
norm_inf =  7.0


array([[0, 0, 1],
       [1, 1, 1],
       [4, 2, 1]])

Обусловленность:
$$(A+\delta A)u = f + \delta f$$
$$\frac{||\delta u||}{||u||}\le \frac{\mu}{1-\mu\frac{||\delta A||}{||A||}} \left(\frac{||\delta f||}{||f||}+\frac{||\delta A||}{||A||}\right)$$

$\mu$ - число обусловленности матрицы A, $\mu(A) = ||A^{-1}||\cdot||A||$, $\mu \ge 1$.



## Пример проблемы использования метода Гаусса для решения СЛАУ

In [2]:
import numpy as np
import numpy.linalg as la

def gauss( A, b ):
    n = b.size
    for k in range(0,n-1):
        for i in range(k+1,n):
            if A[i,k]!=0:
                c = A[i,k]/A[k,k]
                A[i,k+1:n] = A[i,k+1:n] - c*A[k,k+1:n]
                b[i] = b[i] - c*b[k]
                
    # обратный ход
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(A[k,k+1:n],b[k+1:n]))/A[k,k];
    return b

#все числа в представлены как вещественные
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]]);
b1 = np.array([0., 0., 1.]);

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

In [3]:
print(A1)

[[ 1.e-16  1.e+00 -1.e+00]
 [-1.e+00  2.e+00 -1.e+00]
 [ 2.e+00 -1.e+00  0.e+00]]


In [4]:
print(A2)

[[ 2.e+00 -1.e+00  0.e+00]
 [-1.e+00  2.e+00 -1.e+00]
 [ 1.e-16  1.e+00 -1.e+00]]


In [5]:
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]]);
b1 = np.array([0., 0., 1.]);

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

print('mu1 = ', la.cond(A1))
print('mu2 = ', la.cond(A2))

print('u1 = ', gauss(A1, b1))
#print('u1 = ', la.solve(A1, b1))
print('u2 = ', gauss(A2, b2))#la.solve(A2, b2))

mu1 =  16.393731622284385
mu2 =  16.393731622284392
u1 =  [0.55511151 0.25       0.25      ]
u2 =  [1. 1. 1.]


## Часть 1. LU разложение

Задание: 

реализовать алгоритм решения предыдущей задачи с матрицей A2 с помощью LU-разложение В решении должна выводиться L, U и собственно решение системы. 

ВАЖНО: реализация метода LU должна быть получена путем небольшой модификации метода gauss!  При это саму реализацию можно разделить на два метода: один метод собственно находит LU разложение (можно сделать переделкой цикла для матрицы A метода gauss), второй метод - непосредственное решение системы с помощью прямого и обратного хода. Ни в каком виде нельзя ползоваться пакетными методами (в частности, la.solve)

#### 1) Алгоритм разложения матрицы А в LU

In [125]:
import numpy as np
def LU(A):
    # Проверка существования LU разложения из курса Вычебры, все верхние 
    # подматрицы нашей матрицы - невырождены
    n = A.shape[0]
    for i in range(n):
        if np.linalg.det(A[:i+1, :i+1]) == 0:
            raise Exception('Извините, не можем разложить это в LU')
    L = np.zeros([A.shape[0], A.shape[1]])
    for i in range(n):
        L[i, i] = 1
    U = np.zeros([A.shape[0], A.shape[1]])
    
    for i in range(n):
        for j in range(n):
            if i <= j:
                U[i, j] = A[i, j] - np.dot(L[i, :i], U[:i, j])
            if i > j:
                L[i, j] = (A[i, j] - np.dot(L[i, :j], U[:j, j])) / U[j, j]
                 
            
    return L, U

In [126]:
A = np.matrix([[5,-3],[4,1]])
sol = LU(A)
display(sol[0])
display(sol[1])
print('-----------Проверка-----------')
display(np.dot(sol[0],sol[1]))

array([[1. , 0. ],
       [0.8, 1. ]])

array([[ 5. , -3. ],
       [ 0. ,  3.4]])

-----------Проверка-----------


array([[ 5., -3.],
       [ 4.,  1.]])

#### 2) Алгоритм решения СЛУ через LU разложение

In [127]:

def LU_sys_sol(L, U, b):
    n = L.shape[0]
    y = np.matrix(np.zeros([n, 1]))
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i],y[:i]) # прямая подстановка с исользованием нижней треугольной L
                                       # Ly = b 
    x = np.matrix(np.zeros([n, 1]))
    for i in reversed(range(n)):
        x[i] = (y[i] - np.dot(U[i, i:],x[i:])) / U[i, i] # обратная подстановка с исользованием верхней треугольной U
                                                   # Ux = y
    return x

In [128]:
b = np.matrix([1, 11]).T
x = (LU_sys_sol(sol[0], sol[1], b))
print('x =', x)
print('-----------Проверка-----------')
print(np.dot(A, x))

x = [[2.]
 [3.]]
-----------Проверка-----------
[[ 1.]
 [11.]]


### LU - разложение с помощью пакета sympy

Чтобы убедиться, что разложение получено верно, можно воспользоваться скриптом ниже

In [129]:
import sympy as sp

A = sp.Matrix([[5, -3], [4, 1]])
L, U, _ = A.LUdecomposition()
display(L)
display(U)
L*U

Matrix([
[  1, 0],
[4/5, 1]])

Matrix([
[5,   -3],
[0, 17/5]])

Matrix([
[5, -3],
[4,  1]])

## Часть 2. Нахождение обратной матрицы с помощью LU разложения

Задание:

Предложить алгоритм с использованием LU-разложения и найти обратную матрицу с точностью $\epsilon = 10^{-3}$:
$$
A = \begin{pmatrix} 
1 & 1  & 1 \\
0 & 1 & 2 \\
7 & 1 & 4 \\
\end{pmatrix}
$$
Для необходимых оценок использовать первую норму. Сравнить результат со значением, найденным с помощью функции numpy.inv.

#### 3) Алгоритм нахождения обратной матрицы с помощью LU разложения 

In [130]:
import numpy.linalg as la
# Ипользуем LU разложение как метод решения системы Аx = E_i, где E_i - i-й столбец единичной матрицы
# Соответственно из всех таких столбцов х - составим обратную матрицу к А, окторая при умножении на неё даст Е
def LU_mat_inv(L, U):
    n = A.shape[0]
    answ = np.zeros((n,n))
    E = np.identity(n)
    for i in range(n):    
        x = LU_sys_sol(L, U, E[i]) 
        for j in range(n):
            answ[j][i] = x[j]
    return answ

In [131]:
A = np.matrix([[1, 1, 1],[0, 1, 2], [7, 1, 4]])
LU_res = LU(A)
A_inv = LU_mat_inv(LU_res[0], LU_res[1])
print(A_inv)
print(' ')
print('-----------Проверка-----------')
print(' ')
print(la.inv(A))

[[ 0.22222222 -0.33333333  0.11111111]
 [ 1.55555556 -0.33333333 -0.22222222]
 [-0.77777778  0.66666667  0.11111111]]
 
-----------Проверка-----------
 
[[ 0.22222222 -0.33333333  0.11111111]
 [ 1.55555556 -0.33333333 -0.22222222]
 [-0.77777778  0.66666667  0.11111111]]
